# Import Necessary Libraries

In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

from dask.distributed import Client, LocalCluster
from statsmodels.tsa.stattools import adfuller

%matplotlib inline

# Loading Data

In [86]:
df = dd.read_csv("C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022.parquet.csv")
df.compute().head()

,tpep_pickup_datetime,passenger_count,VendorID,PULocationID
0,2022-03-31 15:00:00,4,4,4.0
1,2022-03-31 16:00:00,3,3,3.0
2,2022-03-31 17:00:00,0,0,0.0
3,2022-03-31 18:00:00,0,0,0.0
4,2022-03-31 19:00:00,0,0,0.0


In [87]:
df.compute().tail()

,tpep_pickup_datetime,passenger_count,VendorID,PULocationID
21619,2022-09-30 19:00:00,12107,8576,8576.0
21620,2022-09-30 20:00:00,9959,6905,6905.0
21621,2022-09-30 21:00:00,10046,6894,6894.0
21622,2022-09-30 22:00:00,10244,6999,6999.0
21623,2022-09-30 23:00:00,9633,6547,6547.0


In [88]:
df.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21624 entries, 0 to 21623
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tpep_pickup_datetime  21624 non-null  string 
 1   passenger_count       21624 non-null  int64  
 2   VendorID              21624 non-null  int64  
 3   PULocationID          21624 non-null  float64
dtypes: float64(1), int64(2), string(1)
memory usage: 992.6 KB


In [89]:
df.compute().describe()

,passenger_count,VendorID,PULocationID
count,21624.000000,21624.000000,21624.000000
mean,2549.634758,1833.866260,1833.866260
std,3826.749732,2745.905933,2745.905933
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,5685.000000,4103.250000,4103.250000
max,14770.000000,10527.000000,10527.000000


In [90]:
df.tpep_pickup_datetime = dd.to_datetime(df.tpep_pickup_datetime)
df.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21624 entries, 0 to 21623
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   tpep_pickup_datetime  21624 non-null  datetime64[ns]
 1   passenger_count       21624 non-null  int64         
 2   VendorID              21624 non-null  int64         
 3   PULocationID          21624 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 675.9 KB


In [91]:
df.compute().loc[0, 'tpep_pickup_datetime'].weekofyear

13

In [92]:
df.compute().loc[0, 'tpep_pickup_datetime'].dayofyear

90

In [93]:
######## Day, Month, Hour

In [94]:
data = df.set_index('tpep_pickup_datetime').copy()
data.compute()

,passenger_count,VendorID,PULocationID
tpep_pickup_datetime,,,
2022-01-01 00:00:00,6838,4261,4261.0
2022-01-01 01:00:00,7738,4997,4997.0
2022-01-01 02:00:00,5974,3901,3901.0
2022-01-01 03:00:00,4107,2686,2686.0
2022-01-01 04:00:00,2485,1651,1651.0
...,...,...,...
2023-04-18 10:00:00,0,0,0.0
2023-04-18 11:00:00,0,0,0.0
2023-04-18 12:00:00,0,0,0.0


In [95]:
data.index.month.compute()

Index([1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       ...
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype='int32', name='tpep_pickup_datetime', length=21624)

In [100]:
def add_timed_features(df: pd.DataFrame) -> pd.DataFrame:
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['dayofweek'] = df.index.dayofweek
    return df

In [104]:
# add_timed_features(data)